# 4 GUI

In [1]:
import pyaudio
import wave
def start_audio(time = 5,save_file="01_data_user/genium/test2.wav"):
    print("start recording!")
    
#     CHUNK = 1024
#     FORMAT = pyaudio.paInt16
#     CHANNELS = 2
#     RATE = 16000
#     RECORD_SECONDS = time  #需要录制的时间
#     WAVE_OUTPUT_FILENAME = save_file #保存的文件名
#     p = pyaudio.PyAudio() #初始化
    
#     print("ON")

#     stream = p.open(format=FORMAT,
#                     channels=CHANNELS,
#                     rate=RATE,
#                     input=True,
#                     frames_per_buffer=CHUNK)#创建录音文件
#     frames = []

#     for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#         data = stream.read(CHUNK)
#         frames.append(data)#开始录音
#     stream.stop_stream()
#     stream.close()
#     p.terminate()

#     print("OFF")
    
#     wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')	#保存
#     wf.setnchannels(CHANNELS)
#     wf.setsampwidth(p.get_sample_size(FORMAT))
#     wf.setframerate(RATE)
#     wf.writeframes(b''.join(frames))
#     wf.close()
    pass


In [2]:
import tkinter as tk
import os
import librosa
import math
import json

# DATASET_PATH = "res/Data/genres_reduced"
DATASET_PATH = "01_data_user"
# JSON_PATH = "data.json"
JSON_PATH = "data_add_user.json"
SAMPLE_RATE = 22050
DURATION = 5 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION


def save_mfcc(dataset_path, json_path, n_mfcc = 13, n_fft = 2048, hop_length = 512, num_segments = 5):
    
    # dictionary to store data
#     data = {
#         "mapping" : ["classical", "blues"],
#         "mfcc" : [[], [], []],
#         "labels" : [0, 0, 1]
#     }
    data = {
        "mapping" : [],
        "mfcc" : [],
        "labels" : []
    }
    
    num_samples_per_segments = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segments / hop_length) # 1.2 -> 2
    
    # loop through all the genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        # ensure that we're not at the root level
        if dirpath is not dataset_path:
            
            # save the semantic label
            dirpath_components = dirpath.split("/") # genre/blues => ["genre", "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
#             print("\nProcessing {}".format(semantic_label))
            
            # process files for a specific genre
            for f in filenames:
                
                # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr = SAMPLE_RATE)
                
                # process segments extracting mfcc and storing data
                for s in range(num_segments):
                    start_sample = num_samples_per_segments * s # s = 0 -> 0
                    finish_sample = start_sample + num_samples_per_segments # s = 0 -> num_samples_per_segments
                    
                    mfcc = librosa.feature.mfcc(y = signal[start_sample:finish_sample],
                                                sr = sr,
                                                n_fft = n_fft,
                                                n_mfcc = n_mfcc,
                                                hop_length = hop_length
                                                )
                    
                    mfcc = mfcc.T
                    
                    
                    # store mfcc for segment if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i - 1) # first i is data_path itself
#                         print("{}, segment: {}".format(file_path, s))
                    
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent = 4)

In [3]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

DATA_PATH = "data_add_user.json"

def load_data(data_path):
    """Loads training dataset from json file
    
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    
    """
    with open(data_path, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

def prepare_datasets(test_size, validation_size):
    
    # load data
    X, y = load_data(DATA_PATH)
    
    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)
    
    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = validation_size)
    
    return (X_train, X_validation, X_test, y_train, y_validation, y_test)

def build_model(input_shape):
    """
    Generates RNN-LSTM model
    
    :param input_shape (tuple): Shape of input set
    :return model: RNN-LSTM model
    """
    
    # create model
    model = keras.Sequential()
    
    # 3 LSTM layers
    model.add(keras.layers.LSTM(512, input_shape = input_shape, return_sequences = True))
    model.add(keras.layers.LSTM(256))
    
    # dense layers
    model.add(keras.layers.Dense(128, activation = 'relu'))
    model.add(keras.layers.Dropout(0.3))
    # dense layers
    model.add(keras.layers.Dense(32, activation = 'relu'))
    model.add(keras.layers.Dropout(0.3))
    
    # output layer
    model.add(keras.layers.Dense(2, activation = 'softmax'))
    
    return model
    
    
def plot_history(history):
    
    fig, axs = plt.subplots(2)
    plt.figure(dpi=450)
    # create accuracy subplot
    axs[0].plot(history.history["accuracy"], label = "train accuracy")
    axs[0].plot(history.history["val_accuracy"], label = "test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc = "lower right")
    axs[0].set_title("Accuracy eval")
    
    # create accuracy subplot
    axs[1].plot(history.history["loss"], label = "train error")
    axs[1].plot(history.history["val_loss"], label = "test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc = "upper right")
    axs[1].set_title("Error eval")
    
    plt.show()
    
def predict(model, X, y):
    
    X = X[np.newaxis, ...]
    
    # prediction = [ [0.1, 0.2, ...]]
    prediction = model.predict(X) # X -> (1, 130, 13, 1)
    
    # extract index with max value
    predicted_index = np.argmax(prediction, axis = 1) #[4]
    print("Expected index: {}, Predicted index: {}".format(y,  predicted_index))

In [8]:
# print("start detecting!")
save_mfcc(DATASET_PATH, JSON_PATH)
model=keras.models.load_model('02_model/1205_5s.h5')
# load data
X, y = load_data(DATA_PATH)
y_pre_prob = model.predict(X)
y_pre = np.argmax(y_pre_prob, axis = 1)
y_pre_c = [0 if x > y - 0.9999999 else 1 for x, y in y_pre_prob]

if(all(y_pre_id == 1 for y_pre_id in y_pre_c)):
    print("You are a real human!")

else:
    print("You're probably a robot!")

/home/pshower/miniconda3/envs/Fintech/lib/python3.9/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


You are a real human!


In [9]:
y_pre

array([1])

In [10]:
y_pre_c

[1]

In [11]:
y_pre_prob

array([[2.781276e-10, 1.000000e+00]], dtype=float32)

In [6]:
y_pre_c = [0 if x > y - 0.9999999 else 1 for x, y in y_pre_prob]

In [7]:
y_pre_c

[0, 1, 0, 1, 0]

In [5]:
y_pre

array([1, 1, 1, 1, 0])

In [9]:
y_pre_prob

array([[3.7004186e-13, 1.0000000e+00],
       [2.3388203e-13, 1.0000000e+00],
       [4.8844693e-09, 1.0000000e+00],
       [2.4617463e-04, 9.9975377e-01],
       [1.5485071e-09, 1.0000000e+00]], dtype=float32)

In [4]:
def detect_audio():
    print("start detecting!")
    save_mfcc(DATASET_PATH, JSON_PATH)
    model=keras.models.load_model('02_model/1205_5s.h5')
    # load data
    X, y = load_data(DATA_PATH)
    y_pre_prob = model.predict(X)
    y_pre = np.argmax(y_pre_prob, axis = 1)
    
    if(all(y_pre_id == 1 for y_pre_id in y_pre)):
        print("You are a real human!")
        
    else:
        print("You're probably a robot!")

In [ ]:
import tkinter as tk
import sys

class ExampleApp(tk.Tk):
    def __init__(self):
        super(ExampleApp, self).__init__()
        self.title("Vioce Detect")
        
        toolBar = tk.Frame(self)
        toolBar.pack(side=tk.TOP, fill=tk.X)

        button1 = tk.Button(self, text='Record', command=self.rec)
        button2 = tk.Button(self, text='Detect', command=self.det)
        button1.pack(in_=toolBar, side=tk.LEFT)
        button2.pack(in_=toolBar, side=tk.LEFT)

        self.text = tk.Text(self, wrap='word')
        self.text.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        self.text.tag_configure('stderr', foreground='#b22222')

        sys.stdout = TextRedirector(self.text, 'stdout')
        sys.stderr = TextRedirector(self.text, 'stderr')

    def rec(self):
        start_audio()

    def det(self):
        detect_audio()

class TextRedirector(object):
    def __init__(self, widget, tag='stdout'):
        self.widget = widget
        self.tag = tag

    def write(self, str):
        self.widget.configure(state='normal')
        self.widget.insert(tk.END, str, (self.tag,))    # (self.tag,) 是设置配置
        self.widget.configure(state='disabled')

if __name__ == '__main__':
    app = ExampleApp()
    app.mainloop()


2024-12-07 10:51:15.876106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-07 10:51:15.896242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-07 10:51:15.896823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-07 10:51:15.897827: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [ ]:
import tkinter as tk
import sys

class ExampleApp(tk.Tk):
    def __init__(self):
        super(ExampleApp, self).__init__()
        self.title("Voice Detect")
        
        # 创建工具栏框架，并使用Grid布局
        toolBar = tk.Frame(self)
        toolBar.pack(side=tk.TOP, fill=tk.X)
        
        # 创建按钮并添加到工具栏
        button1 = tk.Button(toolBar, text='Record', command=self.rec, padx=10, pady=5)
        button2 = tk.Button(toolBar, text='Detect', command=self.det, padx=10, pady=5)
        button1.grid(row=0, column=0, padx=10, pady=5)
        button2.grid(row=0, column=1, padx=10, pady=5)

        # 创建滚动条和文本框
        self.scrollbar = tk.Scrollbar(self)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.text = tk.Text(self, wrap='word', yscrollcommand=self.scrollbar.set)
        self.text.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        self.scrollbar.config(command=self.text.yview)
        
        self.text.tag_configure('stderr', foreground='#b22222')

        sys.stdout = TextRedirector(self.text, 'stdout')
        sys.stderr = TextRedirector(self.text, 'stderr')

    def rec(self):
        try:
            start_audio()
        except Exception as e:
            sys.stderr.write(f"Error starting audio: {e}\n")

    def det(self):
        try:
            detect_audio()
        except Exception as e:
            sys.stderr.write(f"Error detecting audio: {e}\n")

class TextRedirector(object):
    def __init__(self, widget, tag='stdout'):
        self.widget = widget
        self.tag = tag

    def write(self, str):
        self.widget.configure(state='normal')
        self.widget.insert(tk.END, str, (self.tag,))
        self.widget.configure(state='disabled')
        self.widget.see(tk.END)  # 自动滚动到文本框的底部

if __name__ == '__main__':
    app = ExampleApp()
    app.mainloop()


In [ ]:
# Create the main window
root = tk.Tk()
root.title("Vioce Detect")
 
# Create a label widget
label = tk.Label(root, text="I can detect whether you are a robot!")

 
# # 创建Tkinter窗口
# window = tk.Tk()
 

# # 创建文本框用于显示输出
# output_text = tk.Text(window)
# output_text.pack()
 
# # 重定向标准输出到文本框
# redirect_stdout_to_tkinter(output_text)
 
# Pack the label into the main window
label.pack(pady=10)
 
# Create a record
button = tk.Button(root, text="Record your voice", command=start_audio)
 
# Pack the button into the main window
button.pack(pady=10)

# Create a button widget
button = tk.Button(root, text="Result", command=detect_audio)
 
# Pack the button into the main window
button.pack(pady=10)
 
# Start the Tkinter event loop
root.mainloop()

In [11]:
import tkinter as tk
from tkinter import messagebox

# 创建主窗口
root = tk.Tk()
root.title('演示窗口')
root.geometry('300x100+630+80')

# 创建按钮
btn1 = tk.Button(root, text="点击")
btn1.pack()

# 定义点击事件
def test(e):
    messagebox.showinfo("窗口名称", "点击成功")

# 绑定事件
btn1.bind("<Button-1>", test)

# 进入消息循环
root.mainloop()